In [795]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

<img src = './docs/1.png' height ='800px'>

In [796]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

#Create a ansatz V with n qubits
def V(num_qubits: int):
    return ansatz.stargraph (num_qubits=num_qubits)

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calRho3 (rho, unitary_matrix, kraus_operators):
    rho2 = sum(K @ rho @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    rho3 = (np.transpose(np.conjugate(unitary_matrix)) @ rho2 @ unitary_matrix)
    return rho3

def createKraus(unitary_matrix):
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #print(Q.shape)
    for q in Q:
        
        q = np.expand_dims(q, 1)
        #print(q)
        #print(np.transpose(np.conjugate(q)))
        #print(q @ np.transpose(np.conjugate(q)))
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))

    
    return tf.convert_to_tensor(kraus_ops)

def tf_sqrtm(matrix):
    # Eigenvalue Decomposition to compute the matrix square root
    eigenvalues, eigenvectors = tf.linalg.eigh(matrix)
    sqrt_eigenvalues = tf.sqrt(tf.maximum(eigenvalues, 0))  # Ensure non-negative eigenvalues
    sqrt_matrix = tf.matmul(eigenvectors, tf.linalg.diag(sqrt_eigenvalues))
    sqrt_matrix = tf.matmul(sqrt_matrix, tf.linalg.adjoint(eigenvectors))
    return sqrt_matrix

import spicy
def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """

    return tf.linalg.trace(
            rho
            @ (tf.linalg.sqrtm(sigma))
        )
    


In [797]:
num_qubits = 2

#Create a ansatz V with n qubits
circuit = V(num_qubits)

#Assign random parameter
num_params = circuit.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)
circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))

print(circuit)

     ┌────────────┐   
q_0: ┤ Ry(5.5279) ├─■─
     ├────────────┤ │ 
q_1: ┤ Ry(4.9312) ├─■─
     └────────────┘   


In [798]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data
print (unitary_matrix.shape) #(16, 16 as num_qubits is 4)

(4, 4)


In [799]:
KrausOperators = createKraus(unitary_matrix=unitary_matrix)
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))


tf.Tensor(
[[1.00000000e+00+0.j 2.77555756e-17+0.j 1.11022302e-16+0.j
  0.00000000e+00+0.j]
 [2.77555756e-17+0.j 1.00000000e+00+0.j 2.77555756e-17+0.j
  0.00000000e+00+0.j]
 [1.11022302e-16+0.j 2.77555756e-17+0.j 1.00000000e+00+0.j
  0.00000000e+00+0.j]
 [0.00000000e+00+0.j 0.00000000e+00+0.j 0.00000000e+00+0.j
  1.00000000e+00+0.j]], shape=(4, 4), dtype=complex128)


In [800]:
#rho
rho = DensityMatrix.from_label('0' * num_qubits)
rho = tf.convert_to_tensor(rho)
rho3 = calRho3(rho=rho, kraus_operators=KrausOperators, unitary_matrix=unitary_matrix)
print(rho3)
print(rho)


tf.Tensor(
[[ 0.26237234+0.j  0.0803281 +0.j -0.05308222+0.j  0.07430116+0.j]
 [ 0.0803281 +0.j  0.18791558+0.j  0.12718158+0.j  0.08827562+0.j]
 [-0.05308222+0.j  0.12718158+0.j  0.42254783+0.j  0.10992494+0.j]
 [ 0.07430116+0.j  0.08827562+0.j  0.10992494+0.j  0.12716426+0.j]], shape=(4, 4), dtype=complex128)
tf.Tensor(
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]], shape=(4, 4), dtype=complex128)


In [801]:
#cost func compare rho, rho3
def cost(rho, rho3):
    return 1-compilation_trace_fidelity(rho, rho3)

print(cost(rho=rho, rho3=rho3))

#Nhận U, K trả về rho3

tf.Tensor((0.508462845496223+0j), shape=(), dtype=complex128)


In [802]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

<img src = './docs/2.png' height ='800px'>

<img src = './docs/3.jpg' height ='800px'>

<img src = './docs/4.png' height ='200px'>

In [803]:
"""def Derivative(rho, unitary_matrix, kraus_operators, epsilon=0.01, alpha=0.1):
    # Deep copy of the Kraus operators to avoid unintended modifications
    kraus_operators_plus = kraus_operators.copy()
    kraus_operators_minus = kraus_operators.copy()
    c = []
    
    # Loop through Kraus operators
    for n, k in enumerate(kraus_operators):
        k_plus = k + epsilon # Kj(+epsilon)
        k_minus = k - epsilon # Kj(-epsilon)
      
        kraus_operators_plus[n] = k_plus # Replace Kj by new Kj_plus
        kraus_operators_minus[n] = k_minus # Replace Kj by new Kj_minus

        # Compute fidelity for K+ and K-
        fil_plus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_plus))
        fil_minus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_minus))
        
        # Compute the derivative
        derivative = -(fil_plus - fil_minus) / epsilon

        # Accumulate
        c.append(derivative)
        
        # Reset Kraus operators for next iteration
        kraus_operators_plus[n] = k
        kraus_operators_minus[n] = k
    
    # Convert c to numpy array for further calculations
    c = np.array(c)
    
    # Compute the projection term
    kraus_operators_conj_transpose = np.transpose(np.conjugate(kraus_operators))
    proj = c - kraus_operators @ (kraus_operators @ np.transpose(np.conjugate(c)) + kraus_operators_conj_transpose @ c) / 2
    
    # Print debugging information
    print('c=', c)
    print('proj=', proj)
    
    # Return the updated Kraus operators
    return kraus_operators - proj"""

"def Derivative(rho, unitary_matrix, kraus_operators, epsilon=0.01, alpha=0.1):\n    # Deep copy of the Kraus operators to avoid unintended modifications\n    kraus_operators_plus = kraus_operators.copy()\n    kraus_operators_minus = kraus_operators.copy()\n    c = []\n    \n    # Loop through Kraus operators\n    for n, k in enumerate(kraus_operators):\n        k_plus = k + epsilon # Kj(+epsilon)\n        k_minus = k - epsilon # Kj(-epsilon)\n      \n        kraus_operators_plus[n] = k_plus # Replace Kj by new Kj_plus\n        kraus_operators_minus[n] = k_minus # Replace Kj by new Kj_minus\n\n        # Compute fidelity for K+ and K-\n        fil_plus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_plus))\n        fil_minus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_minus))\n        \n        # Compute the derivative\n        derivative 

In [804]:
#Auto Diff
def Derivative(rho, unitary_matrix, kraus_operators, epsilon=0.01, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        y = calRho3(rho, unitary_matrix, tensorKraus)
        f = compilation_trace_fidelity(rho, y)
        #f = tf.convert_to_tensor(f)
    
    #print("RHO= ", rho)
    #print("RHO3= ", y)
    # Get the gradient of y with respect to x
    c = tape.gradient(f, tensorKraus)

    #print("Function value:", f)
    #print("Gradient: ", c)

    # Convert c to numpy array for further calculations
    c = np.array(c[:,:, 0])
    
    # Compute the projection term
    kraus_operators_conj_transpose = np.transpose(np.conjugate(kraus_operators))
    proj = c - kraus_operators @ (kraus_operators @ np.transpose(np.conjugate(c)) + kraus_operators_conj_transpose @ c) / 2
    
    # Print debugging information
    print('c=', c)
    print('proj=', proj)
    
    # Return the updated Kraus operators
    return kraus_operators - proj



In [805]:
import tensorflow as tf
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x**2
        
# Get the gradient of y with respect to x
c = tape.gradient(y, x)
print (y)
print("Function value:", y.numpy())
print("Gradient: ", c.numpy())

tf.Tensor(9.0, shape=(), dtype=float32)
Function value: 9.0
Gradient:  6.0


In [806]:
KrausOperatorsTry = tf.identity(KrausOperators)

In [816]:
# try looping manually
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry))
a = cost(rho, calRho3(rho, unitary_matrix, KrausOperatorsTry))
KrausOperatorsTry=Derivative(rho, unitary_matrix, KrausOperatorsTry) 
print (a)

tf.Tensor(
[[ 14.18606766+0.j  15.51753278+0.j  38.76430119+0.j  17.95577833+0.j]
 [ 15.51753278+0.j  21.14653177+0.j  48.67686923+0.j  21.70139177+0.j]
 [ 38.76430119+0.j  48.67686923+0.j 118.55366201+0.j  53.92487103+0.j]
 [ 17.95577833+0.j  21.70139177+0.j  53.92487103+0.j  26.44010672+0.j]], shape=(4, 4), dtype=complex128)
c= [[ 0.09916279+0.j -0.04273983+0.j -0.09838151+0.j -0.02406772+0.j]
 [ 0.18768393+0.j -0.04457908+0.j -0.07207834+0.j -0.03735972+0.j]
 [ 0.34588026+0.j -0.07471752+0.j -0.11650536+0.j -0.06503191+0.j]
 [ 0.29729137+0.j -0.06508895+0.j -0.105995  +0.j -0.05437943+0.j]]
proj= tf.Tensor(
[[[ 1.28870352e+00+0.j -2.11847338e-01+0.j -3.59050655e-01+0.j
   -9.75187934e-02+0.j]
  [ 1.35332049e+00+0.j -2.30139378e-01+0.j -3.60443117e-01+0.j
   -1.40855941e-01+0.j]
  [ 3.21157872e+00+0.j -5.46239818e-01+0.j -8.51022351e-01+0.j
   -3.43792212e-01+0.j]
  [ 1.66453487e+00+0.j -2.82615572e-01+0.j -4.43972806e-01+0.j
   -1.75595921e-01+0.j]]

 [[ 8.10017128e-02+0.j -8.605715